In [ ]:
from sympy import factorint
import re

test_p = '7/11 715/14 935/21 1/7 2/13 3/17' # 126 -> 2250
test_p = '1/2 1/3'
factorint_1 = lambda n: {1:1} if n == 1 else factorint(n)
merg_add =  lambda A, B: {x: A.get(x, 0) + B.get(x, 0) for x in set(A).union(B)}
merg_sub =  lambda A, B: {x: A.get(x, 0) - B.get(x, 0) for x in set(A).union(B)}

def parse_prog(s):
    s = s.replace('^', '**')
    arr = [i for i in re.split(',| ', s) if i != '']
    prog = []
    for i in arr:
        t = tuple(map(factorint_1, map(eval, re.split('/', i))))
        prog.append(t)
    return prog

def exec_fractran(prog, inp):
    inp = factorint_1(inp)
    t = 0
    count = 0
    check = 0

    while 1:
        t += 1
        if t == len(prog):
            if check == 0:
                break
            if check == 1:
                t = 0
                check = 0

        count += 1

        current = prog[t]
 
        if current[1] == {1:1}:
            print(1, current, inp)
            inp = merg_add(inp, current[0])
            t += 1
            print(1, current, inp)
            check = 1
            continue
        
        if current[0] == {1:1}:
           
            if all(inp.get(k, 0) >= current[1][k] for k in current[1]):
                print(2, current, inp)
                inp = merg_sub(inp, current[1])
                t += 1
                print(2, current, inp)
                check = 1
                continue

        if all(inp.get(k, 0) >= current[1][k] for k in current[1]):
            print(3, current, inp)

            inp = merg_add(inp, current[0])
            inp = merg_sub(inp, current[1])

            print(3, current, inp)
            check = 1
            continue

       

    print(count)
    return inp

p = parse_prog(test_p)

exec_fractran(p, 8)

1


{2: 3}

In [ ]:
from sympy import factorint
import re

factorize = lambda n: [] if n == 1 else [i for i in factorint(n).items()]
test_p = '7/11 715/14 935/21 1/7 2/13 3/17' # 126 -> 2250

def parse_prog(s):
    s = s.replace('^', '**')
    arr = [i for i in re.split(',| ', s) if i != '']
    prog = []
    for i in arr:
        t = tuple(map(factorize, map(eval, re.split('/', i))))
        prog.append(t)
    return prog


def check(state, denom):
  """Checks if the program has at least the specified exponents for each prime."""
  for p, val in denom:
    if state[p] < val:
      return False
  return True

def update_state(state, num, denom):
  """Checks the program's state and updates it according to an instruction."""
  if check(state, denom):
    for p, val in denom:
      state[p] -= val
    for p, val in num:
      state[p] += val
    return True
  else:
    return False

def format_state(state):
  return dict((i, v) for i, v in enumerate(state) if v != 0)

def make_usage_map(program, maxidx):
  firstref = [len(program)] * maxidx
  for i, (num, denom) in enumerate(program):
    for idx, value in denom:
      if firstref[idx] == len(program):
        firstref[idx] = i
  return firstref

def make_jump_map(program, firstref):
  jump_map = []
  for i, (num, denom) in enumerate(program):
    if num:
      jump_map.append(min(min(firstref[idx] for idx, val in num), i))
    else:
      jump_map.append(i)
  return jump_map

def fractran(program, input, debug_when=None):
  """Executes a Fractran program and returns the state at the end."""
  maxidx = max(z[0] for instr in program for part in instr for z in part) + 1
  state = [0]*maxidx

  if isinstance(input, int):
    input = factorize(input)

  for prime, val in input:
    state[prime] = val

  firstref = make_usage_map(program, maxidx)
  jump_map = make_jump_map(program, firstref)

  pc = 0
  length = len(program)
  while pc < length:
    num, denom = program[pc]
    if update_state(state, num, denom):
      if num:
        pc = jump_map[pc]
      if debug_when and debug_when(state):
        print(format_state(state))
    else:
      pc += 1

  return format_state(state)


N = 2**2 * 3**5 * 7

pr = parse_prog(test_p)

fractran(pr, N, lambda x:1)


{2: 1, 3: 5, 5: 1, 11: 1, 13: 1}
{2: 1, 3: 5, 5: 1, 7: 1, 13: 1}
{3: 5, 5: 2, 11: 1, 13: 2}
{3: 5, 5: 2, 7: 1, 13: 2}
{3: 4, 5: 3, 11: 1, 13: 2, 17: 1}
{3: 4, 5: 3, 7: 1, 13: 2, 17: 1}
{3: 3, 5: 4, 11: 1, 13: 2, 17: 2}
{3: 3, 5: 4, 7: 1, 13: 2, 17: 2}
{3: 2, 5: 5, 11: 1, 13: 2, 17: 3}
{3: 2, 5: 5, 7: 1, 13: 2, 17: 3}
{3: 1, 5: 6, 11: 1, 13: 2, 17: 4}
{3: 1, 5: 6, 7: 1, 13: 2, 17: 4}
{5: 7, 11: 1, 13: 2, 17: 5}
{5: 7, 7: 1, 13: 2, 17: 5}
{5: 7, 13: 2, 17: 5}
{2: 1, 5: 7, 13: 1, 17: 5}
{2: 2, 5: 7, 17: 5}
{2: 2, 3: 1, 5: 7, 17: 4}
{2: 2, 3: 2, 5: 7, 17: 3}
{2: 2, 3: 3, 5: 7, 17: 2}
{2: 2, 3: 4, 5: 7, 17: 1}
{2: 2, 3: 5, 5: 7}


{2: 2, 3: 5, 5: 7}